In [13]:
from dotenv import load_dotenv
import os


load_dotenv(dotenv_path=".env")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENVIRONMENT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [14]:
# LLM
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
output = model.invoke("自己紹介してください")
print(output)



私は、山田太郎と申します。東京都出身で、現在は大学生として都内の大学に通っています。趣味はスポーツ観戦や音楽鑑賞で、特にサッカーやロックバンドが好きです。将来の夢は、国際的な企業で働くことで、留学経験も積んで自分を磨きたいと思っています。また、人とのコミュニケーションを大切にし、常に新しいことに挑戦することで成長していきたいと考えています。よろしくお願いします。


In [15]:
# Chat Model
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います！よろしくお願いします！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前が分かりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

はい、あなたの名前はジョンさんですね！他に何かお話ししたいことがありますか？


In [16]:
messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは!"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

こんにちは！どのようにお手伝いできますか？

In [17]:
# PromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""以下の料理のレシピを考えてください。
                                      
料理名: {dish}""")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

text='以下の料理のレシピを考えてください。\n\n料理名: カレー'


In [18]:
# MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}")
    ]
)
prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは!私はジョンと言います!よろしくお願いします!"),
            AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前が分かりますか？"
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは!私はジョンと言います!よろしくお願いします!', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]


In [19]:
# PydanticOutputParserを使ったPythonオブジェクトへの変換
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [20]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [21]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)
prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
カレー


In [23]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、全体をよく混ぜる。",
    "水を加え、煮立ったらアクを取り除く。",
    "弱火にして、蓋をして約20分煮る。",
    "カレールーを加え、よく溶かしてさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


In [24]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体をよく混ぜる。', '水を加え、煮立ったらアクを取り除く。', '弱火にして、蓋をして約20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


In [25]:
# StrOutputParser
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
output = output_parser.invoke(ai_message)
print(type(output))
print(output)

<class 'str'>
こんにちは。私はAIアシスタントです。


In [26]:
# LangChain Expression Language (LCEL)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [27]:
chain = prompt | model

In [28]:
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message.content)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら火を止めます。

6. **味を調える**:
   - お好みで塩や胡椒で味を調整します。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- ほうれん草のソテー

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて楽しんでください！おいしいカレーをお楽しみください。


In [29]:
# StrOutputParserを連鎖に追加
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(type(output))
print(output)

<class 'str'>
カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら火を止めます。

6. **味を調える**:
   - お好みでガーリックパウダーや生姜を加え、味を調整します。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて完成です。お好みで福神漬けやらっきょうを添えても良いでしょう。

### ポイント
- 野菜はお好みで他のもの（ピーマン、ナスなど）を加えても美味しいです。
- カレーは時間が経つほど味がなじむので、作り置きにも最適です。

ぜひお試しください！


In [30]:
# PydanticOutputParserを連鎖に追加

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [32]:
chain = prompt_with_format_instructions | model | output_parser

In [33]:
recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんとじゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、煮立ったらアクを取り除く。', '弱火にして、蓋をして約20分煮る。', '火を止めてカレールーを加え、よく溶かす。', '再度火を入れ、10分ほど煮込んで完成。']


In [34]:
# Document loader
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

43


In [35]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1379, which is longer than the specified 1000
Created a chunk of size 1455, which is longer than the specified 1000
Created a chunk of size 1299, which is longer than the specified 1000
Created a chunk of size 1422, which is longer than the specified 1000
Created a chunk of size 1174, which is longer than the specified 1000


129


In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.01968870498239994, -0.007562828715890646, 0.029617559164762497, -0.026554401963949203, 0.049940019845962524, 0.025012260302901268, -0.014977781102061272, 0.02129422128200531, 0.025730518624186516, -0.021463222801685333, 0.009273971430957317, -0.010615423321723938, -0.017417743802070618, -0.006010124925523996, -0.011428743600845337, 0.06316440552473068, 0.033335596323013306, -0.00034394499380141497, -0.04495446756482124, 0.02638540044426918, 0.0320047065615654, 0.03532136604189873, -0.03819439932703972, 0.021864602342247963, 0.01868525706231594, -0.018769757822155952, -0.020269649103283882, 0.032237086445093155, -0.00830221176147461, -0.10114755481481552, -0.009332065470516682, -0.0574183464050293, -0.034243982285261154, 0.04677123576402664, -0.023934874683618546, 0.034835487604141235, 0.023343367502093315, 0.013308888301253319, -0.00799589604139328, -0.033758100122213364, 0.00023254245752468705, -0.021009031683206558, 0.021072406321763992, 0.013171575032174587, 0.00648016110062

In [37]:
from langchain_chroma import Chroma

db = Chroma.from_documents(documents=docs, embedding=embeddings)

In [38]:
retriever = db.as_retriever()

In [39]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len={len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata={first_doc.metadata}")
print(first_doc)

len=4
metadata={'source': 'libs/README.md', 'file_path': 'libs/README.md', 'file_type': '.md', 'file_name': 'README.md'}
page_content='* [OpenAI](https://pypi.org/project/langchain-openai/)
* [Anthropic](https://pypi.org/project/langchain-anthropic/)
* [Ollama](https://pypi.org/project/langchain-ollama/)
* [DeepSeek](https://pypi.org/project/langchain-deepseek/)
* [xAI](https://pypi.org/project/langchain-xai/)
* and more

Most integrations have been moved to their own repositories for improved versioning, dependency management, collaboration, and testing. This includes packages from popular providers such as [Google](https://github.com/langchain-ai/langchain-google) and [AWS](https://github.com/langchain-ai/langchain-aws). Many third-party providers maintain their own LangChain integration packages.

For a full list of all LangChain integrations, please refer to the [LangChain Integrations documentation](https://docs.langchain.com/oss/python/integrations/providers).' metadata={'source'

In [40]:
# LCELを使ったRAGのChainの実装
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
    
文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [42]:
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

文脈にはAWSに関連する具体的なDocument loaderについての情報は含まれていませんが、AWSのパッケージがLangChainの統合として存在することが示されています。AWSのS3からデータを読み込むためのDocument loaderがあるかどうかは、LangChainのAWS統合パッケージのドキュメントを参照することで確認できるでしょう。具体的な情報は、[LangChain Integrations documentation](https://docs.langchain.com/oss/python/integrations/providers)をチェックしてください。
